In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import re
import string
from tqdm import tqdm
import time
from collections import Counter

import nltk
import torch
import spacy
from sklearn.preprocessing import binarize

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split, Subset

import transformers
from transformers import BertTokenizer
import tokenizers

In [ ]:
train_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')
ss_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

train_df.dropna(inplace=True)

In [ ]:
with open('../input/bert-base-uncased/config.json', 'r') as f:
    cfg = f.read()

In [ ]:
cfg

In [ ]:
print('Loading BERT tokenizer...')
PRETRAINED_TOKENIZER = '../input/bert-base-uncased/vocab.txt'
tokenizer = tokenizers.BertWordPieceTokenizer(
        PRETRAINED_TOKENIZER, 
        lowercase=True
    )

In [ ]:
class config:
    MAX_LEN = 64
    TRAIN_BATCH_SIZE = 64
    VALID_BATCH_SIZE = 16
    EPOCHS = 5
    BERT_PATH = "../input/bert-base-uncased/"
    MODEL_PATH = "model.bin"
    TRAINING_FILE = "../input/tweet-train-folds/train_folds.csv"
    TOKENIZER = tokenizers.BertWordPieceTokenizer(
        f"{BERT_PATH}/vocab.txt", 
        lowercase=True
    )

Dataset

In [ ]:
class TweetDataset:
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.selected_text = 'selected_text' in df.columns
        self.max_len = max_len
        
    def __len__(self):
        return len(self.df)
    
    def _prepare_data(self, text, selected_text, sentiment):
        len_selected = len(selected_text)
#         print(f'text={text}')
#         print(f'selected_text={selected_text}')
#         print(f'len_selected={len_selected}')
       
        start_pos = None
        end_pos = None
        for i, txt in enumerate(text):
            if text[i: i + len_selected] == selected_text:
                start_pos = i
                end_pos = i + len_selected - 1
                break
#         print(f'start_pos={start_pos}')
#         print(f'end_pos={end_pos}')
        
        char_targets = [0] * len(text)
        if start_pos != None and end_pos != None:
            for ct in range(start_pos, end_pos + 1):
                char_targets[ct] = 1
#         print(f'char_targets={char_targets}')
        
        text_enc = tokenizer.encode(text)
#         print(f'text_enc={text_enc.tokens}')
        input_ids_orig = text_enc.ids[1:-1]
        CLS_ID = text_enc.ids[0]
        SEP_ID = text_enc.ids[-1]
#         print(f'input_ids={text_enc.ids}')
#         print(f'input_ids_orig={input_ids_orig}')
        text_offsets = text_enc.offsets[1:-1]
#         print(f'text_offsets={text_offsets}')
        
        target_idx = []
        for j, (offset1, offset2) in enumerate(text_offsets):
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)
#         print(f'target_idx={target_idx}')

        targets_start = target_idx[0]
        targets_end = target_idx[-1]
#         print(f'targets_start={targets_start}')
#         print(f'targets_end={targets_end}')
                
        sentiment_id = tokenizer.encode(sentiment).ids[1]
#         print(f'sentiment_id={sentiment_id}')

        input_ids = [CLS_ID] + [sentiment_id] + [SEP_ID] + input_ids_orig + [SEP_ID]
        token_type_ids = [0, 0, 0] + [1] * (len(input_ids_orig) + 1)
        mask = [1] * len(token_type_ids)
        text_offsets = [(0, 0)] * 3 + text_offsets + [(0, 0)]
        targets_start += 3
        targets_end += 3
        
        padding_length = self.max_len - len(input_ids)
        if padding_length > 0:
            input_ids = input_ids + ([0] * padding_length)
            mask = mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
            text_offsets = text_offsets + ([(0, 0)] * padding_length)

        return {
            'ids': input_ids,
            'mask': mask,
            'token_type_ids': token_type_ids,
            'targets_start': targets_start,
            'targets_end': targets_end,
            'orig_text': text,
            'orig_selected': selected_text,
            'sentiment': sentiment,
            'offsets': text_offsets
        }
        
        return text_enc

    def __getitem__(self, item):
        data = process_data(
            self.df.iloc[item].text, 
            self.df.iloc[item].selected_text, 
            self.df.iloc[item].sentiment
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_text': data["orig_text"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }
        
    def __getitem__(self, index):
        text = self.df.loc[index].text
        sel_text = self.df.loc[index].selected_text if self.selected_text else ''
        sentiment = self.df.loc[index].sentiment
        data = self._prepare_data(text, sel_text, sentiment)
        return data
    
    def __len__(self):
        return len(self.df)

In [ ]:
train = TweetDataset(train_df, config.TOKENIZER, config.MAX_LEN)
itr = iter(train)
next(itr)

In [ ]:
train_dataset = TweetDataset(train_df, config.TOKENIZER, config.MAX_LEN)

train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,
    num_workers=1
)

for bi, d in enumerate(train_data_loader):
    ids = d["ids"]
    token_type_ids = d["token_type_ids"]
    mask = d["mask"]
    targets_start = d["targets_start"]
    targets_end = d["targets_end"]
    sentiment = d["sentiment"]
    orig_selected = d["orig_selected"]
    orig_text = d["orig_text"]
    targets_start = d["targets_start"]
    targets_end = d["targets_end"]
    offsets = d["offsets"]
    print(orig_text)
    print(orig_selected)    
    print(sentiment)    
    print(targets_start)
    print(targets_end)    
    break

In [ ]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf, dropout):
        super(TweetModel, self).__init__(conf)
        self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH, config=conf)
        self.drop_out = nn.Dropout(dropout)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )
        print(f'out.shape={out.shape}')
        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.l0(out)
        print(f'logits.shape={logits.shape}')

        start_logits, end_logits = logits.split(1, dim=-1)
        print(f'start_logits.shape={start_logits.shape}')
        print(f'end_logits.shape={end_logits.shape}')

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        print(f'start_logits.shape={start_logits.shape}')
        print(f'end_logits.shape={end_logits.shape}')

        return start_logits, end_logits

In [ ]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    print(f'start_logits={start_logits[0]}')
    print(f'start_positions={start_positions[0]}')
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

In [ ]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    losses = utils.AverageMeter()
    jaccards = utils.AverageMeter()

    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for bi, d in enumerate(tk0):

        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_text = d["orig_text"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)

        model.zero_grad()
        outputs_start, outputs_end = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids,
        )
        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        break
        loss.backward()
        optimizer.step()
        scheduler.step()

        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        jaccard_scores = []
        for px, text in enumerate(orig_text):
            selected_text = orig_selected[px]
            text_sentiment = sentiment[px]
            jaccard_score, _ = calculate_jaccard_score(
                original_text=text,
                target_string=selected_text,
                sentiment_val=text_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            jaccard_scores.append(jaccard_score)

        jaccards.update(np.mean(jaccard_scores), ids.size(0))
        losses.update(loss.item(), ids.size(0))
        tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)